# Interpreter Pattern

The Interpreter pattern defines a grammatical representation for a language and an interpreter to interpret sentences in the language.

## Problem

You need to process and evaluate expressions in a specialized language, grammar, or notation, and the standard approach of parsing and interpretation is too complex or inefficient.

## Solution

Define a class hierarchy representing the grammar rules, where each rule is an expression that can be interpreted. The interpreter recursively evaluates (interprets) the expressions to produce the desired result.

## Example 1: Simple Boolean Expression Interpreter

This example demonstrates a basic interpreter for boolean expressions.

In [31]:
from abc import ABC, abstractmethod


# Abstract Expression
class Expression(ABC):
    @abstractmethod
    def interpret(self, context) -> bool:
        pass

In [32]:
# Terminal Expressions
class VariableExpression(Expression):
    def __init__(self, variable: str):
        self.variable = variable

    def interpret(self, context) -> bool:
        return context.get(self.variable, False)

In [33]:
# Non-terminal Expressions
class AndExpression(Expression):
    def __init__(self, expr1: Expression, expr2: Expression):
        self.expr1 = expr1
        self.expr2 = expr2

    def interpret(self, context) -> bool:
        return self.expr1.interpret(context) and self.expr2.interpret(context)


class OrExpression(Expression):
    def __init__(self, expr1: Expression, expr2: Expression):
        self.expr1 = expr1
        self.expr2 = expr2

    def interpret(self, context) -> bool:
        return self.expr1.interpret(context) or self.expr2.interpret(context)


class NotExpression(Expression):
    def __init__(self, expr: Expression):
        self.expr = expr

    def interpret(self, context) -> bool:
        return not self.expr.interpret(context)

In [34]:
# Context class to store variable values
class Context(dict):
    def __init__(self, variables=None):
        super().__init__()
        if variables:
            for var, value in variables.items():
                self[var] = value

In [35]:
# Client code
if __name__ == "__main__":
    # Define expressions
    # Rule: (A && B) || (!C)
    a = VariableExpression("A")
    b = VariableExpression("B")
    c = VariableExpression("C")

    not_c = NotExpression(c)
    a_and_b = AndExpression(a, b)
    expression = OrExpression(a_and_b, not_c)

    # Create a context
    context1 = Context({"A": True, "B": True, "C": True})
    context2 = Context({"A": True, "B": False, "C": True})
    context3 = Context({"A": True, "B": False, "C": False})

    # Interpret expressions
    print(f"Rule: (A && B) || (!C)")
    print(f"Context: {context1}")
    print(f"Result: {expression.interpret(context1)}\n")

    print(f"Context: {context2}")
    print(f"Result: {expression.interpret(context2)}\n")

    print(f"Context: {context3}")
    print(f"Result: {expression.interpret(context3)}")

Rule: (A && B) || (!C)
Context: {'A': True, 'B': True, 'C': True}
Result: True

Context: {'A': True, 'B': False, 'C': True}
Result: False

Context: {'A': True, 'B': False, 'C': False}
Result: True


## Example 2: Smart Home Command Interpreter with PyParsing

This example demonstrates how to use the pyparsing library to create an interpreter for natural language-like smart home commands.

In [36]:
from pyparsing import Group, OneOrMore, Optional, Suppress, Word, alphanums


class Gate:
    def __init__(self):
        self.is_open = False

    def __str__(self):
        return "open" if self.is_open else "closed"

    def open(self):
        print("opening the gate")
        self.is_open = True

    def close(self):
        print("closing the gate")
        self.is_open = False


class Garage:
    def __init__(self):
        self.is_open = False

    def __str__(self):
        return "open" if self.is_open else "closed"

    def open(self):
        print("opening the garage")
        self.is_open = True

    def close(self):
        print("closing the garage")
        self.is_open = False


class Aircondition:
    def __init__(self):
        self.is_on = False

    def __str__(self):
        return "on" if self.is_on else "off"

    def turn_on(self):
        print("turning on the air condition")
        self.is_on = True

    def turn_off(self):
        print("turning off the air condition")
        self.is_on = False


class Heating:
    def __init__(self):
        self.is_on = False

    def __str__(self):
        return "on" if self.is_on else "off"

    def turn_on(self):
        print("turning on the heating")
        self.is_on = True

    def turn_off(self):
        print("turning off the heating")
        self.is_on = False


class Boiler:
    def __init__(self):
        self.temperature = 83  # in celsius

    def __str__(self):
        return f"boiler temperature: {self.temperature}"

    def increase_temperature(self, amount):
        print(f"increasing the boiler's temperature by {amount} degrees")
        self.temperature += amount

    def decrease_temperature(self, amount):
        print(f"decreasing the boiler's temperature by {amount} degrees")
        self.temperature -= amount


class Fridge:
    def __init__(self):
        self.temperature = 2  # in celsius

    def __str__(self):
        return f"fridge temperature: {self.temperature}"

    def increase_temperature(self, amount):
        print(f"increasing the fridge's temperature by {amount} degrees")
        self.temperature += amount

    def decrease_temperature(self, amount):
        print(f"decreasing the fridge's temperature by {amount} degrees")
        self.temperature -= amount

In [37]:
# Main interpreter function
def main():
    # Define the grammar for our home automation language
    word = Word(alphanums)
    command = Group(OneOrMore(word))
    token = Suppress("->")
    device = Group(OneOrMore(word))
    argument = Group(OneOrMore(word))
    event = command + token + device + Optional(token + argument)

    # Initialize devices
    gate = Gate()
    garage = Garage()
    airco = Aircondition()
    heating = Heating()
    boiler = Boiler()
    fridge = Fridge()

    # Device lookup dictionary
    devices = {
        "gate": gate,
        "garage": garage,
        "air condition": airco,
        "heating": heating,
        "boiler temperature": boiler,
        "fridge temperature": fridge,
    }

    # Test commands
    tests = (
        "open -> gate",
        "close -> garage",
        "turn on -> air condition",
        "turn off -> heating",
        "increase -> boiler temperature -> 5 degrees",
        "decrease -> fridge temperature -> 2 degrees",
    )

    # Define actions available for each device
    open_actions = {
        "gate": gate.open,
        "garage": garage.open,
        "air condition": airco.turn_on,
        "heating": heating.turn_on,
        "boiler temperature": boiler.increase_temperature,
        "fridge temperature": fridge.increase_temperature,
    }
    close_actions = {
        "gate": gate.close,
        "garage": garage.close,
        "air condition": airco.turn_off,
        "heating": heating.turn_off,
        "boiler temperature": boiler.decrease_temperature,
        "fridge temperature": fridge.decrease_temperature,
    }

    # Execute each command
    for t in tests:
        print(f"\nExecuting: '{t}'")
        if len(event.parseString(t)) == 2:
            cmd, dev = event.parseString(t)
            cmd_str, dev_str = " ".join(cmd), " ".join(dev)
            if "open" in cmd_str or "turn on" in cmd_str:
                open_actions[dev_str]()
            elif "close" in cmd_str or "turn off" in cmd_str:
                close_actions[dev_str]()
        elif len(event.parseString(t)) == 3:
            cmd, dev, arg = event.parseString(t)
            cmd_str = " ".join(cmd)
            dev_str = " ".join(dev)
            arg_str = " ".join(arg)
            num_arg = 0
            try:
                # extract the numeric part
                num_arg = int(arg_str.split()[0])
            except ValueError:
                print(f"expected number but got: '{arg_str[0]}'")
            if "increase" in cmd_str and num_arg > 0:
                open_actions[dev_str](num_arg)
            elif "decrease" in cmd_str and num_arg > 0:
                close_actions[dev_str](num_arg)

        # Safe way to get device status
        device_obj = devices[dev_str]
        print(f"Status after command: {dev_str} is {device_obj}")

In [38]:
# Run the interpreter
if __name__ == "__main__":
    main()


Executing: 'open -> gate'
opening the gate
Status after command: gate is open

Executing: 'close -> garage'
closing the garage
Status after command: garage is closed

Executing: 'turn on -> air condition'
turning on the air condition
Status after command: air condition is on

Executing: 'turn off -> heating'
turning off the heating
Status after command: heating is off

Executing: 'increase -> boiler temperature -> 5 degrees'
increasing the boiler's temperature by 5 degrees
Status after command: boiler temperature is boiler temperature: 88

Executing: 'decrease -> fridge temperature -> 2 degrees'
decreasing the fridge's temperature by 2 degrees
Status after command: fridge temperature is fridge temperature: 0


## How This Implements the Interpreter Pattern

In this smart home example:

1. **Language Definition**: The grammar for natural language-like commands is defined using pyparsing components
2. **Parse Tree**: The `event` parser breaks down input strings into structured commands, devices, and arguments
3. **Terminal Expressions**: Individual words and numbers are the terminal elements
4. **Non-terminal Expressions**: Commands, devices, and arguments form the non-terminal expressions
5. **Context**: The various device objects provide the context in which commands are interpreted
6. **Interpretation**: The main function interprets the parsed commands by executing appropriate methods on the target devices

This example shows how the Interpreter pattern can be used to create a simple, domain-specific language for home automation.

## Benefits

* **Domain-Specific Languages**: Ideal for implementing simple domain-specific languages
* **Grammar Extension**: Easy to add new expressions to the language
* **Separation of Grammar and Evaluation**: Grammar rules are decoupled from their interpretation
* **Expressive Power**: Can represent complex rules and expressions in a simple object-oriented way
* **Rule Reusability**: Expression objects can be reused in different contexts
* **Leveraging Parsing Libraries**: Using libraries like pyparsing simplifies complex grammar handling
* **Error Handling**: Proper parsing libraries provide better error reporting and recovery